In [5]:
import sys
import importlib
sys.path.insert(0,'../')
import numpy as np
import netket as nk
from scripts import functions as f
importlib.reload(f)

from scripts import new_dynamics, new_dynamics2
import os
currentpath = os.getcwd()
parentdir = os.path.dirname(currentpath)

In [2]:
from scripts import Dimer_RBM

In [3]:
length=[4, 4]
V = 1.0
h = 1.0
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)


ham = f.dimer_hamiltonian(V = V, h = h ,length=np.array(length))
op_transition1 = f.dimer_flip1(length = np.array(length))


hex_ = nk.machine.new_hex(np.array(length))

ma = nk.machine.RbmDimer(hi, hex_, alpha = 2, symmetry = True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float, reverse=True, half=True)
ma.init_random_parameters(seed=1234)

yoyo
2


In [4]:
ma._ws[:] = 0

In [6]:
ma._set_bare_parameters(
    ma._a, ma._b, ma._w, ma._as, ma._bs, ma._ws, ma._autom, ma._z2
)
op, ad2o_o, op_num, label_num  = f.dimer_flip1(length = np.array(length), return_info = True)

ad2_bool = np.zeros([ad2o_o.shape[0], ad2o_o.shape[0]], dtype = np.bool)
for l in range(ad2o_o.shape[0]):
    label = ad2o_o[l]
    for op_ in label:
        ad2_bool[l,op_] = True
        
ma.hex.ad2o_o = ad2o_o.astype(np.int64)
ma.hex.ad2_bool = ad2_bool

NameError: name 'ma' is not defined

In [4]:

# create save folder.


n_jobs = 4

g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)

op = f.dimer_hamiltonian(h, V,np.array(length))
op_transition = f.dimer_flip1(length = np.array(length))
hex_ = nk.machine.new_hex(np.array(length))




sweep_size = 1000
sa_mul = nk.sampler.DimerMetropolisLocal_multi(machine=ma, op=op_transition, length = length, n_chains=1, sweep_size = sweep_size, kernel = 1, n_jobs=n_jobs)
sa_mul.reset()
sa_mul.generate_samples(1000) # discard the begginings of metropolis sampling.



NameError: name 'nk' is not defined

In [38]:
n_samples_ = 10**4
samples_state = sa_mul.generate_samples(int(n_samples_ / 1))
samples_state = samples_state.reshape(-1, ma.hilbert.size)
print('get samples')

get samples


In [39]:
import scripts.dynamics as Dynamics
import scripts.dynamics_4 as sdy

import scripts
importlib.reload(sdy)

<module 'scripts.dynamics_4' from '../scripts/dynamics_4.py'>

In [40]:
t_list = np.arange(0,10,0.1)

# d = Dynamics.new_dynamics(op, ma)
# %time P_ = d.multiprocess(samples_state, 100, 4) 
# P = P_[0]
# T = P_[1]
# P_list = f.process_P(P, T, t_list)

d = sdy.new_dynamics_one(op, ma)
P_list = d.multiprocess(samples_state, t_list,4) 

In [41]:
edge_corr1 = np.array([ 0.5     ,  0.      ])
edge_corr2 = np.array([ 0.25    , -0.433013])
edge_corr3 = np.array([ 0.75    , -0.433013])

In [42]:
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)
hex_ = nk.machine.new_hex(np.array(length))


x = np.arange(length[0])
y = np.arange(length[1])

xx, yy = np.meshgrid(x,y)
xx = xx.reshape(-1)
yy = yy.reshape(-1)

operators_list = []

for base_edge in [edge_corr1, edge_corr2, edge_corr3]:
    edge_coor_array = hex_.ProcessPeriodic(xx[:,None] * hex_.a1[None,:]+ yy[:,None]  * hex_.a2[None,:] + base_edge)
    edges = hex_.edge_coor_to_lattice(edge_coor_array)
    colors = hex_.get_edge_color(edges)

    operators_list.append(f.return_spin_corr(hi, edges, colors))

In [15]:
operators_list[0][4]._acting_on

array([[12, 16]])

In [16]:
hex_.a2

array([ 0.5       , -0.86602539])

In [17]:
1/2 * np.tan(1/6 * np.pi)

0.28867513459481287

In [18]:
T, color = hex_.autom(reverse=True, half=True)
from numba import njit

In [19]:
@njit
def _permute(x, T, color):
    x_ = np.zeros((T.shape[0], x.shape[-1]))
    
    for j in range(len(x_)):
        for i in range(len(x)):
            x_[j,i] = x[T[j,i]] * color[i]
    return x_

In [20]:
@njit
def permute(X, T, color):
    
    X_ = np.zeros((X.shape[0]*2, T.shape[1], X.shape[-1]))
    
    for a in range(2):
        for i in range(len(X)):
            X_[2*i + a] = _permute(X[i], T[a], color[a])
    return X_.reshape(-1,X.shape[-1])

In [21]:
# new_P_list = np.zeros((P_list.shape[0], P_list.shape[1] * T.shape[0] * T.shape[1], P_list.shape[-1]))

# for t in range(len(new_P_list)):
#     new_P_list[t] = permute(P_list[t], T, color)

In [23]:
dimer_corr = np.zeros((3,3,length[0],length[0],len(t_list)), dtype = np.float64)

for a in range(3):
    for b in range(3):
        dimer_corr[b,a], _, _ = f.cal_dimer_corr(operators_list[a][0], operators_list[b], P_list, hex_, t_list)
        
# for a in range(3):
#     for b in range(a):
#         for i in range(length[0]):
#             for j in range(length[1]):
                
#                 tmp = (dimer_corr[b,a][i,j] + dimer_corr[a,b][-i, -j])/2
#                 dimer_corr[b,a][i,j] = tmp
#                 dimer_corr[a,b][-i, -j] = tmp
                
                
for a in range(3):
    for b in range(a+1):
        for i in range(length[0]):
            for j in range(length[1]):
                
                tmp = (dimer_corr[b,a][i,j] + dimer_corr[a,b][-i,-j])/2
                dimer_corr[b,a][i,j] = tmp
                dimer_corr[a,b][-i,-j] = tmp


In [372]:
dimer_corr[2,1][-1,-1]

array([0.4569, 0.4533, 0.4567, 0.4556, 0.4587, 0.4592, 0.4583, 0.4567,
       0.4549, 0.4534, 0.452 , 0.453 , 0.4521, 0.4537, 0.4502, 0.4484,
       0.4482, 0.4502, 0.446 , 0.4491, 0.452 , 0.45  , 0.4508, 0.444 ,
       0.446 , 0.4456, 0.4488, 0.4485, 0.4436, 0.4422, 0.4393, 0.4425,
       0.4426, 0.4392, 0.443 , 0.4445, 0.4415, 0.4377, 0.4384, 0.4387,
       0.4404, 0.4365, 0.4375, 0.4348, 0.4392, 0.4419, 0.4458, 0.4449,
       0.4468, 0.4484, 0.454 , 0.4484, 0.4486, 0.4471, 0.4485, 0.4466,
       0.4472, 0.4442, 0.444 , 0.4434, 0.4453, 0.4443, 0.4428, 0.4431,
       0.4424, 0.4414, 0.4387, 0.4381, 0.4364, 0.4359, 0.4371, 0.4391,
       0.4405, 0.4365, 0.4411, 0.4392, 0.436 , 0.4394, 0.4397, 0.4418,
       0.4388, 0.4391, 0.4391, 0.4414, 0.4438, 0.4439, 0.4466, 0.4457,
       0.4497, 0.4526, 0.451 , 0.449 , 0.4468, 0.4456, 0.4479, 0.4448,
       0.444 , 0.4437, 0.4429, 0.4436])

In [24]:
from numba import njit
@njit
def dimer_fourier2D(corr):
    
    assert corr.shape[0] == corr.shape[1]
    L = x.shape[0]
    
    momentum = np.zeros((L*2, L*2, corr.shape[-1]), dtype=np.complex128)
    
    for m1 in range(2 * L):
        for m2 in range(2 * L):
            tmp = np.zeros(corr.shape[-1], dtype=np.complex128)
            for n1 in range(L):
                for n2 in range(L):
                    tmp += np.exp(-1j*(n1 * m1/(L) * 2 * np.pi + n2 * m2/(L) * 2 * np.pi)) * corr[n1,n2]
            momentum[m1,m2] = tmp/(2*L)
    
    return momentum

A = np.array([
hex_.a1,
hex_.a2
])
def dimer_fourier2D_sub(dimer_m_):
    assert dimer_m_.shape[0] == dimer_m_.shape[1] == 3
    assert dimer_m_.shape[2] == dimer_m_.shape[3] 
    
    edge_coors = np.array([[ 0.5     ,  0.      ],
                           [ 0.25    , -0.433013],
                           [ 0.75    , -0.433013]])
    [b1, b2] = np.linalg.inv(A.T)
    b = np.array([b1, b2])
    G = edge_coors @ b.T
    
    
    L = dimer_m_.shape[2]
    
    dimer_momentum = np.zeros((L,L,len(t_list)), dtype=np.complex128)
    
    for m1 in range(L):
        for m2 in range(L):
            tmp = np.zeros(dimer_m_.shape[-1],dtype=np.complex128)
            for a in range(3):
                for b in range(3):
                    tmp += np.exp(-1j * (m1* (2*np.pi) /(L) *(G[a,0] - G[b,0]) + m2* (2*np.pi) /(L) *(G[a,1] - G[b,1]))) * dimer_m_[a,b,m1,m2]
            dimer_momentum[m1,m2] = tmp / 3
    return dimer_momentum

In [25]:
dimer_momentum_ = np.zeros((3,3,8,8,len(t_list)),dtype=np.complex128)
for a in range(3):
    for b in range(3):
        dimer_momentum_[a,b] = dimer_fourier2D(dimer_corr[a,b])

In [1]:
import numpy as np
dimer_momentum = np.load('/home/keisuke/Documents/dimermaster/save/corr/h=1.0V=1.0l=[4, 4]/dimer_momentum_mean.npy')

In [3]:
dimer_momentum[0, 0]

array([2.67040598+0.j, 2.67147436+0.j, 2.67013889+0.j, 2.67094017+0.j,
       2.66746795+0.j, 2.66666667+0.j, 2.66693376+0.j, 2.66586538+0.j,
       2.66346154+0.j, 2.65705128+0.j, 2.66826923+0.j, 2.67200855+0.j,
       2.68429487+0.j, 2.6786859 +0.j, 2.6741453 +0.j, 2.67548077+0.j,
       2.67361111+0.j, 2.68082265+0.j, 2.68269231+0.j, 2.68910256+0.j,
       2.68990385+0.j, 2.67922009+0.j, 2.6784188 +0.j, 2.67280983+0.j,
       2.6605235 +0.j, 2.66239316+0.j, 2.65811966+0.j, 2.65037393+0.j,
       2.66105769+0.j, 2.65731838+0.j, 2.65945513+0.j, 2.66346154+0.j,
       2.65731838+0.j, 2.65117521+0.j, 2.65998932+0.j, 2.66399573+0.j,
       2.65892094+0.j, 2.65117521+0.j, 2.66132479+0.j, 2.66826923+0.j,
       2.67174145+0.j, 2.67467949+0.j, 2.67307692+0.j, 2.68856838+0.j,
       2.68429487+0.j, 2.68589744+0.j, 2.68242521+0.j, 2.6829594 +0.j,
       2.67735043+0.j, 2.68215812+0.j, 2.67067308+0.j, 2.6786859 +0.j,
       2.6786859 +0.j, 2.67922009+0.j, 2.67067308+0.j, 2.67574786+0.j,
      

In [36]:
dimer_momentum = dimer_fourier2D_sub(dimer_momentum_)

dimer_momentum[1,3]

array([ 7.35265916e-02+5.80519899e-18j,  6.47960299e-02+1.66187022e-17j,
        5.81618729e-02+1.50285390e-17j,  5.33157985e-02+2.06468219e-17j,
        4.89273487e-02+1.36674627e-17j,  4.38337687e-02+1.37776809e-17j,
        3.87667829e-02+9.83661506e-18j,  3.62093328e-02+1.86223786e-17j,
        3.38536339e-02+1.49844777e-17j,  3.27598826e-02+1.08289687e-17j,
        2.99815721e-02+1.17208234e-17j,  2.86338044e-02+3.85333541e-18j,
        2.73185796e-02+4.21114015e-18j,  2.37240419e-02+7.36085035e-18j,
        2.13812814e-02+2.80600256e-18j,  2.10912619e-02+8.15713060e-18j,
        1.99366524e-02+8.98162970e-18j,  1.94474704e-02+5.52922387e-18j,
        1.91965425e-02+2.16327257e-18j,  1.92137386e-02+2.72374336e-18j,
        1.81806960e-02+4.42785066e-18j,  1.73957433e-02+5.34299961e-18j,
        1.73205564e-02+2.37116286e-18j,  1.58549725e-02+7.91747811e-19j,
        1.42065007e-02+1.66283546e-18j,  1.33265547e-02+1.94210689e-18j,
        1.36545920e-02+1.13970340e-18j,  1.32157743

# 

## import numpy as np

dimer_momentum = np.load("/home/keisuke/Documents/dimermaster/save/corr/h=1.0V=1.0l=[4, 4]/dimer_momentum_mean.npy")

In [359]:
dimer_momentum[1,2]

array([0.04312839-1.68954839e-18j, 0.03887289-4.37746627e-18j,
       0.03537223-4.68917440e-18j, 0.03277867-1.21633931e-18j,
       0.03045852-6.27030256e-18j, 0.02849348-4.56268414e-18j,
       0.02672986-4.94469600e-18j, 0.0249787 -4.17079023e-18j,
       0.02381871-3.61471310e-18j, 0.02253215-3.61061911e-18j,
       0.02146164-9.46418146e-19j, 0.02035038-1.67147835e-18j,
       0.01912295+1.31911264e-18j, 0.01806659-1.45237916e-18j,
       0.01734986+2.14581680e-18j, 0.01657209-5.55653613e-19j,
       0.01592629+1.82281490e-18j, 0.01520482-4.68126876e-19j,
       0.01465175+2.02949094e-18j, 0.0140664 -2.50721752e-19j,
       0.01355227-3.25260652e-19j, 0.01317165-4.69820941e-19j,
       0.01256853-2.07805416e-19j, 0.0122146 -1.95382266e-19j,
       0.01174766-2.86861825e-19j, 0.01119764-1.98770398e-19j,
       0.01090798+2.15089900e-18j, 0.01052176-2.34910471e-19j,
       0.00987125+2.24520200e-18j, 0.00967066-1.26490253e-19j,
       0.00946384+2.17292185e-18j, 0.00886299-1.2649025

In [88]:
tau_list = []
N = 2000
corr1 = []
corr2 = np.zeros((48,N),dtype=np.float64)
n = -1
import re
with open('RN4_4V1.0st.dat') as fe:
    for line in fe:
        
        if re.findall(r'#\d', line):
            i = 0
            n += 1
        else:
            try:
                match = re.findall(r'[\d,\.]+', line)
                tau_list.append(float(match[1]))
                corr2[n,i] = float(match[3])
                i+=1
            except:
                pass

In [65]:
par_path = '/Users/keisukemurota/Documents/Resaerch/Dimer_RBM/save/dat/'
path = par_path + "R[4,4]V1.0.dat"
from scripts import utils
importlib.reload(utils)
utils.write2dat(dimer_momentum, t_list, path)

In [78]:
(corr2[3][:200][None,None,:] - dimer_corr).mean(axis=-1).reshape(-1,order='F')

array([-0.05634956,  0.42758727,  0.00322997,  0.42758727, -0.0441282 ,
        0.42758727,  0.0046754 ,  0.42758727, -0.00115713, -0.00917361,
       -0.01261676, -0.00640277, -0.00570326, -0.0147057 , -0.01300102,
       -0.00211408, -0.02437084,  0.42758727, -0.01894938,  0.42758727,
       -0.02238292,  0.42758727, -0.02108872,  0.42758727, -0.00573926,
       -0.00756955, -0.01115713, -0.01382396, -0.00062284, -0.0066182 ,
       -0.0128662 , -0.01682243, -0.0241016 ,  0.42758727, -0.01379675,
        0.42758727, -0.02562104,  0.42758727, -0.01401078,  0.42758727,
       -0.0072511 , -0.01454327, -0.01759956, -0.00695526, -0.00468229,
       -0.01462088, -0.01405399, -0.00369133, -0.02069845,  0.42758727,
       -0.02181723,  0.42758727, -0.02030079,  0.42758727, -0.02049422,
        0.42758727,  0.00094381, -0.0047047 , -0.01026439, -0.01439945,
       -0.0041022 , -0.00484732, -0.01371034, -0.01257396])